MUSIC RECOMMENDATION SYSTEM
===========================
- ** Predict genres **
- ** Predict moods **
- ** Recommend base on cosin similarity and jaccard similarity **

In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
# makes printing a dict look nice
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [3]:
from collections import namedtuple
# Create a namedtuple called 'Song'
Song = namedtuple("Song", ["artist", "title"])
# I'm choosing to also include probability, so I'm making Mood a namedtuple as well
Mood = namedtuple("Mood", ["description", "probability"])

## Load model

In [47]:
predict_genres = pickle.load(open('predict_genres.pickle', 'rb'))
predict_moods  = pickle.load(open('lyrics_predict_moods.rfc.pickle', 'rb'))
predict_moods_audio = pickle.load(open('audio_predict_moods.rfc.pickle'))
my_database    = pickle.load(open('my_database.pickle', 'rb'))
my_database.reset_index(inplace=True, drop=True)

In [5]:
audio_features = np.array([5,
 0.709932,
 0.231455,
 130.03,
 0.12174099999999999,
 0.036662,
 0.0,
 0,
 4,
 232.46104,
 -5.15,
 0.37439000000000006,
 0.7047289999999999,
 0.5650000000000001,
 0.5650000000000001,
 0.743,
 1.0])
lyrics_features = 'party rock yeah woo let s go party rock is in the house tonight everybody just have a good time and we gon make you loose your mind wooo everybody just have a good time party rock is in the house tonight ooww everybody just have a good time and we gon make you loose your mind yeah we just wanna see you shake that in the club party rock look a pretty girl she on my jock huh non stop when we in the spot booty on the way like she on the block wooo with a drink i gots to know tight jeans tattoos cause i m rock n roll half black half white domino gain the money out the door yoo i m runnin through these hoes like drano i got that devilish flow rock n roll no halo we party rock yeah that s the crew that i m reppin on the rise to the stop no led in our zeppelin heeeey party rock is in the house tonight wooo everybody just have a good time hey and we gon make you loose your mind everybody just have a good time let s go party rock is in the house tonight everybody just have a good time and we gon make you loose your mind we just wanna see you shake that everyday i m shuffling shuffling shuffling step up fast be the first girl to make me throw this cash we gettin money don t be mad now stop hating is bad one more shot for us another round another round please fill up my cup don t mess around bitches wanna see you shake it now now you wanna be you re naked now get up get down put your hands up to the sound get up get down put your hands up to the sound get up get down put your hands up to the sound put your hands up to the sound put your hands up to the sound get up get up get up get up get up get up get up get up get up put your hands up to the sound to the sound put your hands up put your hands up put your hands up put your hands up party rock is in the house tonight put your hands up everybody just have a good time put your hands up and we gon make you loose your mind put your hands up everybody just have a good good good time ooohh ooooh put your hands up ooohh ooooh ooh ooh ooohh ooooh put your hands up ooohh ooooh put your hands up shake that everyday i m shu ff ffling put your put your put your put your yeaah put your put your wooo put your put your your hands up put your hands up put your hands up everyday i m shuffling'

In [6]:
from string import punctuation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.stem.snowball import SnowballStemmer

translator = str.maketrans('','', punctuation)
stemmer = SnowballStemmer('english')

def clean_text(raw_text):
    
    clean_words = []
    
    raw_text = raw_text.lower()
    raw_text = raw_text.translate(translator)
    
    for word in raw_text.split():
        
        if word not in ENGLISH_STOP_WORDS:
            
            clean_words.append(stemmer.stem(word))
    
    return " ".join(clean_words)

In [63]:
# You should submit code that basically looks like this
from sklearn.metrics import jaccard_similarity_score
df_moods = my_database.moods.str.join(',').str.get_dummies(sep=',')
df_genres_moods = pd.concat((my_database.genres, df_moods), axis=1)
def recommend_similar_songs(audio_features, lyrics_features):
    genre = predict_genres.predict(audio_features.reshape((1, -1)))
    lyrics_features_clean = clean_text(lyrics_features)
    moods = predict_moods.predict([lyrics_features_clean])
    predict_result = np.concatenate((genre, moods[0].astype('int').astype('str')))
    similarity = df_genres_moods.apply(lambda x: jaccard_similarity_score(predict_result, x.tolist()), axis=1)
    similarity = similarity.sort_values(ascending=False)[0:50]
    top_3 = my_database.iloc[similarity.sample(3).index]
    result = [ Song(artist=row["artist"], title=row['name']) for idx, row in top_3.iterrows()]
    final_result_dictionary = dict(playlist=result)
    return final_result_dictionary
predict_genres_moods = recommend_similar_songs(audio_features, lyrics_features)
print(predict_genres_moods)

/home/dks/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


{'playlist': [Song(artist='Madeon', title='The City'), Song(artist='Zac Brown Band', title="Whiskey's Gone"), Song(artist='Tiësto', title='United (Ultra Music Festival Anthem)')]}


In [48]:
my_database.head()

,artist,name,audio_features,genres,moods
0,PSY,Gangnam Style (강남스타일),"[11, 0.912744, 0.083704, 132.069, 0.293137, 0....",pop,[energetic]
1,OneRepublic,Counting Stars,"[6, 0.7457039999999999, 0.11995499999999999, 1...",pop,[happy]
2,Carly Rae Jepsen,Call Me Maybe,"[7, 0.585564, 0.10829699999999999, 120.014, 0....",pop,[energetic]
3,One Direction,What Makes You Beautiful,"[4, 0.81403, 0.079196, 124.991, 0.07244, 0.005...",pop,[happy]
4,Katy Perry,Firework,"[8, 0.8695539999999999, 0.107945, 124.065, 0.0...",pop,[happy]


In [49]:
my_database.iloc[2,:]

artist                                             Carly Rae Jepsen
name                                                  Call Me Maybe
audio_features    [7, 0.585564, 0.10829699999999999, 120.014, 0....
genres                                                          pop
moods                                                   [energetic]
Name: 2, dtype: object